# Audio with Transformers

Dans ce notebook, nous allons utiliser la library transformers de Hugging Face pour le traitement de l'audio

## Classification zero-shot

Commençons par un problème déjà vu en vision par ordinateur, la clasification zero-shot. Il va s'agir de donner la provenance d'un extrait sonore sans avoir eu d'entraînement sur des catégories spécifiques.

### Implémentation

Pour cela, nous utilisons le dataset [ESC-50](https://github.com/karolpiczak/ESC-50) (ashraq/esc50) qui regroupe des enregistrements de 5 secondes de 50 classes différentes.   
Pour le télécharger, utilisons la library datasets de Hugging Face : 

In [ ]:
from datasets import load_dataset

# This dataset is a collection of different sounds of 5 seconds
dataset = load_dataset("ashraq/esc50",split="train[0:10]")

On peut observer les métadonnées de l'extrait audio. Le sampling_rate est particulièrement important. Il faut s'assurer que c'est le même que celui des données d'entraînement du modèle.

In [6]:
audio_sample = dataset[0]
audio_sample

{'filename': '1-100038-A-14.wav',
 'fold': 1,
 'target': 14,
 'category': 'chirping_birds',
 'esc10': False,
 'src_file': 100038,
 'take': 'A',
 'audio': {'path': None,
  'array': array([-0.01184082, -0.10336304, -0.14141846, ...,  0.06985474,
          0.04049683,  0.00274658]),
  'sampling_rate': 44100}}

On peut écouter l'extrait audio (attention à ne pas mettre trop fort le son de votre ordinateur) avec IPython.

In [7]:
from IPython.display import Audio as IPythonAudio
IPythonAudio(audio_sample["audio"]["array"],rate=audio_sample["audio"]["sampling_rate"])

Il est temps d'utiliser le pipeline de Hugging Face pour récuperer notre modèle.  
On utilise le modèle [CLAP](https://github.com/LAION-AI/CLAP) de LAION-AI (laion/clap-htsat-unfused).

In [9]:
from transformers import pipeline
audio_zero_shot = pipeline(task="zero-shot-audio-classification",model="laion/clap-htsat-unfused")

Regardons le sampling rate du modèle pour voir si il correspond  celui de nos données.

In [14]:
print("Sampling rate du modèle : ",audio_zero_shot.feature_extractor.sampling_rate)
print("Sampling rate de notre extrait : ",audio_sample["audio"]["sampling_rate"])

Sampling rate du modèle :  48000
Sampling rate de notre extrait :  44100


On va devoir modifier le sampling rate de notre dataset pour l'adapter au modèle.

In [16]:
from datasets import Audio
dataset = dataset.cast_column("audio",Audio(sampling_rate=48_000))
audio_sample = dataset[0]
print("Sampling rate de notre extrait : ",audio_sample["audio"]["sampling_rate"])

Sampling rate de notre extrait :  48000


Maintenant que les extraits et le modèle sont sur la même longueur d'onde, on va pouvoir procéder à notre classification.  
On va proposer des labels candidats (comme pour le modèle CLIP en vision).

In [18]:
candidate_labels = ["Sound of a dog","Sound of cat"]
outputs=audio_zero_shot(audio_sample["audio"]["array"],candidate_labels=candidate_labels)
print("Score de "+candidate_labels[0],outputs[0]["score"])
print("Score de "+candidate_labels[1],outputs[1]["score"])

Score de Sound of a dog 0.9805886149406433
Score de Sound of cat 0.019411340355873108


Le modèle est capable d'identifier que l'extrait audio est un aboiement de chien et pas un miaulement de chat.   
Vous pouvez tester sur vos propres extraits sonores ou sur d'autres extraits du dataset.